# Autoenkoderi

U ovoj svesci biće prikazano kako se autoenkoderi mogu kreirati korišćenjem objektne podrške Keras biblioteke.

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras import backend as K

Specifični model dobija se nasleđivanjem `Model` objekta biblioteke Keras. Slojevi koje model objedinjuje se kreiraju u konstruktorskoj metodi `__init__`, a propagacija unapred kroz mrežu se definiše metodom `call`. 

Model koji ćemo kreirati odgovara autoenkoderu konstruisanom u prethodnom primeru. 

In [3]:
class Autoencoder(Model):
    def __init__(self, latent_dim, kernel_size, layer_filters):
        super(Autoencoder, self).__init__()
        
        self.image_size = 28
        
        self.latent_dim = latent_dim
        self.kernel_size = kernel_size
        self.layer_filters = layer_filters
        
        self.encoder = self.__create_encoder(self.image_size)
        
        self.decoder = self.__create_decoder(self.image_size)
        
    def __create_encoder(self, image_size):
        input_shape = (image_size, image_size, 1)
        inputs = Input(shape=input_shape, name='encoder_input')
        x = inputs
        for filters in self.layer_filters:
            x = Conv2D(filters=filters,
                   kernel_size=self.kernel_size,
                   strides=2,
                   activation='relu',
                   padding='same')(x)
        self.shape = K.int_shape(x)
        x = Flatten()(x)
        latent_outputs = Dense(latent_dim, name='latent_vector')(x)
        
        return Model(inputs=inputs, outputs=latent_outputs, name='encoder')
        
    def __create_decoder(self, image_size):
        latent_inputs = Input(shape=(self.latent_dim,), name='decoder_input')
        
        shape = self.shape
        x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
        x = Reshape((shape[1], shape[2], shape[3]))(x)
        
        for filters in self.layer_filters[::-1]:
            x = Conv2DTranspose(filters=filters,
                            kernel_size=self.kernel_size,
                            strides=2,
                            activation='relu',
                            padding='same')(x)

        x = Conv2DTranspose(filters=1,
                    kernel_size=self.kernel_size,
                    padding='same')(x)
        
        outputs = Activation('sigmoid', name='decoder_output')(x)

        return Model(inputs=latent_inputs, outputs=outputs, name='decoder')
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [4]:
latent_dim = 16
kernel_size = 3
layer_filters = [32, 64]

In [5]:
autoencoder = Autoencoder(latent_dim, kernel_size, layer_filters)

Samo kompajliranje modela, treniranje i testiranje mreže se sprovesti na uobičajen način tj. korišćenjem `compile`, `fit` i `predict` funkcija.